In [ ]:
MODEL_NAME = "n_beats_biggysmall"

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from multiprocessing.dummy import freeze_support
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['WANDB_NOTEBOOK_NAME'] = 'n-beats.ipynb'
os.environ['WANDB_API_KEY'] = os.getenv('WANDB_API_KEY')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score, rmse

from darts import TimeSeries

from darts.datasets import EnergyDataset

import helper
import glob

from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(project="Digital-Energy", name=MODEL_NAME, log_model=True)


AVAILABLE_GPUS = torch.cuda.device_count()
AVAILABLE_CPUS = os.cpu_count()

print(f"Available GPUs: {AVAILABLE_GPUS}")
print(f"Available CPUs: {AVAILABLE_CPUS}")


# Data

Taking the first 300 households from the London Dataset and converting them to a Darts TimeSeries.

In [ ]:
my_time_series_dataset = []
for x in sorted(glob.glob("../../Data/london_clean/*.csv"))[:1000]:
    df = pd.read_csv(f'{x}')
    df["DateTime"] = pd.to_datetime(df['DateTime'])
    #df = df.groupby(pd.Grouper(key='DateTime', freq='1D')).max("KWHhh").round(3).reset_index()
    series = TimeSeries.from_dataframe(df, time_col='DateTime', value_cols='KWHhh').astype(np.float32)
    my_time_series_dataset.append(series)


In [ ]:
## sets
training_sets = []
validation_sets = []
for x in my_time_series_dataset:
    train, val = series.split_after(0.90)
    training_sets.append(train)
    validation_sets.append(val)

# Model

We create a N-Beats model that utilizes the GPU, Weights, Biases logger and early stopping callback.

## Early stopping

An early stopping callback is used to stop the training if the validation loss does not improve after a certain number of epochs.


In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.01,
    patience=10,
    verbose=True,
    mode="min"
    )

In [ ]:
encoders = {
    # "datetime_attribute": {"future": ["DateTime"], "past": ["DateTime"]},
    "position": {"past": ["absolute"], "future": ["relative"]},
    "transformer": Scaler(),
}

In [ ]:
# input chunk = The length of the input sequence fed to the model
# output chunk = The length of the output sequence predicted by the model
model_nbeats = NBEATSModel(
    input_chunk_length=96,
    output_chunk_length=1,
    generic_architecture=False,
    #num_stacks=10,
    num_blocks=3,
    num_layers=5,
    layer_widths=512,
    n_epochs=40,
    nr_epochs_val_period=1,
    batch_size=2048,
    work_dir="../../Models",
    save_checkpoints=True,
    model_name=MODEL_NAME,
    pl_trainer_kwargs={
    "enable_progress_bar": True,
    "enable_model_summary": True,
    "accelerator": "gpu",
    "devices": 1,
    "logger": wandb_logger,
    "callbacks": [early_stop_callback]
    },
    #loss_fn=torch.nn.CrossEntropyLoss() custom loss function
    # optimizer_cls=torch.optim.Adam,
    log_tensorboard=True,
    add_encoders=encoders,
)

In [ ]:
#wandb_logger.watch(model_nbeats) # sadly this feature does not work for Darts models
model_nbeats.fit(series=training_sets, val_series=validation_sets, num_loader_workers=AVAILABLE_CPUS)

In [ ]:
START = 3000
for i, x in enumerate(glob.glob("../../Data/london_clean/*.csv")[START:START+10]):

    df = pd.read_csv(x)
    df["DateTime"] = pd.to_datetime(df['DateTime'])
    series = TimeSeries.from_dataframe(df, value_cols=['KWHhh'], time_col="DateTime", fill_missing_dates=True, freq="30min").astype(np.float32)
    series = series[-500:]


    pred_series = model_nbeats.historical_forecasts(
        series,
        forecast_horizon=1,
        stride=1,
        retrain=False,
        verbose=True,
    )

    print(f"rmse: {rmse(series, pred_series)}.")
    print(f"R2 score: {r2_score(series, pred_series)}.")

    helper.display_forecast(pred_series, series, "1 day", save=True, fig_name=f"{i}-test", model_name=f"{MODEL_NAME}", fig_size=(20,10))


# Loading checkpoints of the model

loading the best checkpoint of the model. To compare the results of the model with the previous one.

In [ ]:
# load the model
model_nbeats = NBEATSModel.load_from_checkpoint(work_dir="../../Models/", model_name=MODEL_NAME)

In [ ]:
helper.display_forecast(pred_series, series, "1 day", save=False, fig_name=f"test", model_name=f"{MODEL_NAME}", fig_size=(20,10))